In [94]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from collections import OrderedDict

In [95]:
transform = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

dataset = MNIST(".", train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [96]:
# Check the quantity of the dataset
total_data = 0

for image, label in dataloader:
    total_data += image.shape[0]

print("total quantity of the dataset # {} ".format(total_data))
print("The shape of the data # {} ".format(image.shape))

total quantity of the dataset # 60000 
The shape of the data # torch.Size([32, 1, 64, 64]) 


In [97]:
class ResidualBlock(nn.Module):
    """
    A ResidualBlock module used in deep neural networks to enable training of deeper networks.
    Consists of two sets of Convolution, BatchNorm, and PReLU layers. The input is added to the output
    of these layers, creating a residual connection.

    Parameters:
        in_channels (int): The number of channels in the input feature map.

    Forward pass:
        Accepts a 4D Tensor as input and returns a 4D Tensor with the same dimensions.
        Applies the residual block operations and adds the input to the output of these operations.
    """
    def __init__(self, in_channels = 64):
        self.in_channels = in_channels
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(
                in_channels=self.in_channels, out_channels=self.in_channels, kernel_size=3, stride=1,padding=1,),
            nn.BatchNorm2d(num_features=self.in_channels),
            nn.PReLU(),
            
            nn.Conv2d(
                in_channels = self.in_channels, out_channels=self.in_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=in_channels),
        )

    def forward(self, x):
        return x + self.block(x)


class UpSample(nn.Module):
    """
    An UpSample module to upscale an image by a factor of 2 using PixelShuffle.
    Uses a Conv2D layer to increase the number of channels followed by PixelShuffle
    to upscale the image and a PReLU activation.

    Forward pass:
        Accepts a 4D Tensor as input and returns a 4D Tensor with the spatial dimensions
        increased by a factor of 2 and the channels reduced by a factor of 4.
    """
    def __init__(self):
        super(UpSample, self).__init__()
        self.upsample = nn.Sequential(
            nn.Conv2d(
                in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1
            ),
            nn.PixelShuffle(upscale_factor=2),
            nn.PReLU(),
        )

    def forward(self, x):
        return self.upsample(x)


class Generator(nn.Module):
    """
    Generator module of a Generative Adversarial Network (GAN) designed for tasks like image super-resolution.
    It consists of an initial convolution, a series of ResidualBlocks, followed by Upsampling blocks,
    and a final convolutional layer.

    Parameters:
        in_channels (int): The number of channels in the input images.
        num_residual_blocks (int): The number of ResidualBlocks in the network.

    Forward pass:
        Accepts a 4D Tensor as input and passes it through the initial layer, a series of ResidualBlocks,
        upsampling blocks, and a final convolutional layer to produce a 4D Tensor as output.
        The output tensor has the same number of channels as the input and increased spatial dimensions.
    """
    def __init__(self, in_channels=1, num_residual_blocks=16):
        self.in_channels = in_channels
        self.num_residual_block = num_residual_blocks
        super(Generator, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels = self.in_channels, out_channels=64, kernel_size=9, stride=1, padding=4),
            nn.PReLU()
        )

        self.residuals = nn.Sequential(
            *[ResidualBlock(64) for _ in range(num_residual_blocks)]
        )

        self.upsample = nn.Sequential(*[UpSample() for _ in range(2)])

        self.final = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=in_channels, kernel_size=9, stride=1, padding=4), nn.Tanh()
        )

    def forward(self, x):
        initial = self.initial(x)
        residuals = self.residuals(initial)
        x = initial + residuals
        x = self.upsample(x)
        x = self.final(x)
        return x

In [98]:
if __name__ == "__main__":
    generator = Generator()

In [99]:
# Find out the total trainable parameters
total_params = 0
for params in generator.parameters():
    total_params+=params.numel()
    
print("Total # of trainable parameters # {}".format(total_params))

Total # of trainable parameters # 1491668


In [100]:
# Check it works or not
noise_data = torch.randn(64, 1, 64, 64)
generator(noise_data).shape

torch.Size([64, 1, 256, 256])

In [101]:
class Discriminator(nn.Module):
    """
    Discriminator module of a Generative Adversarial Network (GAN) used for discriminating between real and generated images.
    The architecture consists of a series of convolutional layers with varying kernel sizes and strides,
    followed by Batch Normalization and LeakyReLU activation. The network ends with an Adaptive Average Pooling layer
    and two convolutional layers reducing the output to a single scalar value representing the probability
    that the input image is real.

    Parameters:
        in_channels (int): The number of channels in the input images.

    The architecture is dynamic and can be configured through the layers_config attribute,
    which defines the structure of convolutional layers in the network.

    Forward pass:
        Accepts a 4D Tensor as input and passes it through a series of convolutional,
        Batch Normalization, and LeakyReLU layers. The output is then passed through an Adaptive Average Pooling layer
        and two more convolutional layers to produce a single scalar value as output,
        representing the probability that the input image is real.
    """
    def __init__(self, in_channels=1):
        self.in_channels = in_channels
        super(Discriminator, self).__init__()
        self.nf = 64

        self.layers_config = [
            (self.in_channels, self.nf, 3, 1, 1, 0.2),
            (self.nf, self.nf, 3, 2, 1, 0.2),
            (self.nf, self.nf * 2, 3, 1, 1, 0.2),
            (self.nf * 2, self.nf * 2, 3, 2, 1, 0.2),
            (self.nf * 2, self.nf * 4, 3, 1, 1, 0.2),
            (self.nf * 4, self.nf * 4, 3, 2, 1, 0.2),
            (self.nf * 4, self.nf * 8, 3, 1, 1, 0.2),
            (self.nf * 8, self.nf * 8, 3, 2, 1, 0.2),
            
        ]
        self.model = self.connected_layer(layers_config=self.layers_config)

        self.main = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels=self.nf * 8, out_channels=1024, kernel_size=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=1),
            nn.Sigmoid(),
        )

    def connected_layer(self, layers_config):
        layers = OrderedDict()
        if layers_config:
            for index, (
                in_channels, out_channels, kernel_size, stride, padding, negative_slope) in enumerate(layers_config):
                layers["{}_conv".format(index)] = nn.Conv2d(in_channels = in_channels,
                                                            out_channels = out_channels,
                                                            kernel_size = kernel_size,
                                                            stride = stride, padding = padding)
                layers["{}_batch_norms".format(index)] = nn.BatchNorm2d(
                    num_features = out_channels)
                layers["{}_activation".format(index)] = nn.LeakyReLU(
                    negative_slope = negative_slope, inplace = True)

            return nn.Sequential(layers)

    def forward(self, x):
        x = self.model(x)
        x = self.main(x)
        return x.view(-1, 1)

In [102]:
if __name__ == '__main__':
    discriminator = Discriminator()
    

In [103]:
noise_data = torch.randn(64, 1, 64, 64)
discriminator(noise_data).shape

torch.Size([64, 1])

In [104]:
# Total trainable params
total_params = 0
for params in discriminator.parameters():
    total_params+=params.numel()
    
print("Total trainable params # {} ".format(total_params))

Total trainable params # 5214401 
